In [1]:
import sys
from config import *

Starting with Spotify API
Caution!!!

Spotify API has a limit of the number of calls made every 30 seconds. If you exceed this limit, you will get a "429" error code and you will have to wait several hours to be able to make new requests.

Be extremelly carefull to avoid this problem.

In [2]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentias #
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret= client_secret))


Create a function to search a given single song in the Spotify API: search_song(title, artist, limit).

In [3]:
def search_song(title, artist, limit):
    
    '''
    function to search a given single song in the Spotify API: search_song(title, artist, limit).
      
    '''
    list_of_ids = []
    results =sp.search(q="tracks:"+title, limit= limit)
    if results['tracks']['items']:
        for item in results['tracks']['items']:
            #print("The name of song is: '{}' and the id is: {}".format(item['name'],item['id']))
            list_of_ids.append(item['id'])

    else:
        print("Song not found")
        list_of_ids.append('NONE')
    return list_of_ids



Create a function get_audio_features(list_of_song_ids) to obtain the audio features of a given list of songs (the content of list_of_songs can be the href/id/uri or a list with a single song IDs).

In [4]:
import time
import numpy as np
def  get_audio_features(list_of_song_ids):
    
    ''' this function will  obtain the audio features of a given list of songs with given ids'''
    #split the list of song IDs into "chunks" of 50 song IDs
    
    chunks = np.array_split(list_of_song_ids,50)
    chunk_count = len(chunks)
    print("Total Chunk Count:", chunk_count)
    
    song_audio_features = []    
    for index, chunk in enumerate(chunks):
        print(chunk)
        print("Getting the audio features for chunk: ",index+1)
        
        list_of_audio_features = sp.audio_features(chunk)
        song_audio_features += list_of_audio_features
        time.sleep(20)
    return (pd.DataFrame(song_audio_features))

In [5]:
import pandas as pd
def add_audio_features(df,audio_features_df):
    # concat songs dataframe with audio features
    df_with_audio_features = pd.concat([df, audio_features_df], axis=1)
    display(df_with_audio_features.head())
# saving concat file
    return(df_with_audio_features)

creating dataframe with song name, artists with id

In [20]:
import pandas as pd
#read top_100 file
top_100 = pd.read_csv("top_100.csv")
#display(top_100.head())
list_of_ids=[]
for song in top_100['Songs']:
    try:
        id = search_song(song, "", 1)    #search song in spotify get id
        list_of_ids.append(id)           #add ids to list
    except:
        print("Song not found!")
        list_of_ids.append("NONE")

#add id column to dataframe        
top_100["id"] = list_of_ids
display(top_100.head())    #display
top_100=top_100[top_100['id']!="NONE"]     # delete rows with none id
#pd.to_csv("top_100_with_id.csv")
top_100.to_csv(r"./top_100_with_id.csv", index=False)   #savig dataframe with song name , artist and id csv

,Songs,Artists,id
0,Paint The Town Red,Doja Cat,[2IGMVunIBsBLtEQyoI1Mu7]
1,Snooze,SZA,[7HNAsAfrpuY7kSbi1Kuk29]
2,Fast Car,Luke Combs,[1ZWEehHLyzfkYn0jVslHCy]
3,Cruel Summer,Taylor Swift,[2EGaDf0cPX789H3LNeB03D]
4,I Remember Everything,Zach Bryan Featuring Kacey Musgraves,[7pjPeCHhB5gvwONDGvNtH2]


creating dataframe with song id and audio features

In [21]:
list_of_ids

[['2IGMVunIBsBLtEQyoI1Mu7'],
 ['7HNAsAfrpuY7kSbi1Kuk29'],
 ['1ZWEehHLyzfkYn0jVslHCy'],
 ['2EGaDf0cPX789H3LNeB03D'],
 ['7pjPeCHhB5gvwONDGvNtH2'],
 ['1nX9KhK3Fff27SnrIor2Yb'],
 ['3HotlaWARUUyzVbgJdig2b'],
 ['4rXLjWdF2ZZpXCVTfWcshS'],
 ['5I30ByMf5nhgbSokY1EZiN'],
 ['2FiHYjZn3IMbkPCnhcyBqT'],
 ['741UUVE2kuITl0c6zuqqbO'],
 ['0AYt6NMyyLd0rLuvr0UkMH'],
 ['0bILU0UiNDmA5Ff6whhb2R'],
 ['1vhbThkFnqAHbZp0fEjXE4'],
 ['5ZQMnWe2T0UGJZt5DAjLvV'],
 ['0Aj0pG8cJNjZK8zO2Wtgm8'],
 ['4offR0dV5c6dcIvEkduNdv'],
 ['388itNxxzYI90pBa6OZHXj'],
 ['0BlYwEaERbk7ox1IqIhHhc'],
 ['5twSy6ffBrF03mnNUsUgJZ'],
 ['0V3wPSX9ygBnCm8psDIegu'],
 ['41WQUSINanQHfhfKpFDsms'],
 ['78Du4CMFgnhdlG33gblkiP'],
 ['4Ie43xzNjvedClTOHYYp19'],
 ['6B8wwJQs2PsT5ITBGtA5CZ'],
 ['6pRRHgDjfIGhTUCGlHX2p8'],
 ['22VHOlVYBqytsrAqV8yXBK'],
 ['3wdACrQHBkIajKwglqJogE'],
 ['6KaKmX6EIu3x2057HoZwQF'],
 ['5ROhanbAv4DowIN50SYgNX'],
 ['1u4Mi9bV0BTjb8kRBRyvWJ'],
 ['1O8zCu3dLLPxkbGfIsbRgE'],
 ['1yW00C89gUWPvUh5fkyzz4'],
 ['22WrpYvNFAcdSEAZeaVPm6'],
 ['27GmP9AWRs7

In [22]:
list_of_ids = [ item[0] for item in list_of_ids]
list_of_ids

['2IGMVunIBsBLtEQyoI1Mu7',
 '7HNAsAfrpuY7kSbi1Kuk29',
 '1ZWEehHLyzfkYn0jVslHCy',
 '2EGaDf0cPX789H3LNeB03D',
 '7pjPeCHhB5gvwONDGvNtH2',
 '1nX9KhK3Fff27SnrIor2Yb',
 '3HotlaWARUUyzVbgJdig2b',
 '4rXLjWdF2ZZpXCVTfWcshS',
 '5I30ByMf5nhgbSokY1EZiN',
 '2FiHYjZn3IMbkPCnhcyBqT',
 '741UUVE2kuITl0c6zuqqbO',
 '0AYt6NMyyLd0rLuvr0UkMH',
 '0bILU0UiNDmA5Ff6whhb2R',
 '1vhbThkFnqAHbZp0fEjXE4',
 '5ZQMnWe2T0UGJZt5DAjLvV',
 '0Aj0pG8cJNjZK8zO2Wtgm8',
 '4offR0dV5c6dcIvEkduNdv',
 '388itNxxzYI90pBa6OZHXj',
 '0BlYwEaERbk7ox1IqIhHhc',
 '5twSy6ffBrF03mnNUsUgJZ',
 '0V3wPSX9ygBnCm8psDIegu',
 '41WQUSINanQHfhfKpFDsms',
 '78Du4CMFgnhdlG33gblkiP',
 '4Ie43xzNjvedClTOHYYp19',
 '6B8wwJQs2PsT5ITBGtA5CZ',
 '6pRRHgDjfIGhTUCGlHX2p8',
 '22VHOlVYBqytsrAqV8yXBK',
 '3wdACrQHBkIajKwglqJogE',
 '6KaKmX6EIu3x2057HoZwQF',
 '5ROhanbAv4DowIN50SYgNX',
 '1u4Mi9bV0BTjb8kRBRyvWJ',
 '1O8zCu3dLLPxkbGfIsbRgE',
 '1yW00C89gUWPvUh5fkyzz4',
 '22WrpYvNFAcdSEAZeaVPm6',
 '27GmP9AWRs744SzKcpJsTZ',
 '0ubl4gpohoq5KcQiqI5lUe',
 '5i3TnSdOFNX7xdb0X6YU7V',
 

In [23]:
import numpy as np
#get audio features using ids
df_audio_features_top_100 = get_audio_features(list_of_ids)
display(df_audio_features_top_100.head())
#saving dataframe to csv file having id and audio features
df_audio_features_top_100.to_csv(r"./df_audio_features_top_100.csv", index=False) 

Total Chunk Count: 50
['2IGMVunIBsBLtEQyoI1Mu7' '7HNAsAfrpuY7kSbi1Kuk29']
Getting the audio features for chunk:  1
['1ZWEehHLyzfkYn0jVslHCy' '2EGaDf0cPX789H3LNeB03D']
Getting the audio features for chunk:  2
['7pjPeCHhB5gvwONDGvNtH2' '1nX9KhK3Fff27SnrIor2Yb']
Getting the audio features for chunk:  3
['3HotlaWARUUyzVbgJdig2b' '4rXLjWdF2ZZpXCVTfWcshS']
Getting the audio features for chunk:  4
['5I30ByMf5nhgbSokY1EZiN' '2FiHYjZn3IMbkPCnhcyBqT']
Getting the audio features for chunk:  5
['741UUVE2kuITl0c6zuqqbO' '0AYt6NMyyLd0rLuvr0UkMH']
Getting the audio features for chunk:  6
['0bILU0UiNDmA5Ff6whhb2R' '1vhbThkFnqAHbZp0fEjXE4']
Getting the audio features for chunk:  7
['5ZQMnWe2T0UGJZt5DAjLvV' '0Aj0pG8cJNjZK8zO2Wtgm8']
Getting the audio features for chunk:  8
['4offR0dV5c6dcIvEkduNdv' '388itNxxzYI90pBa6OZHXj']
Getting the audio features for chunk:  9
['0BlYwEaERbk7ox1IqIhHhc' '5twSy6ffBrF03mnNUsUgJZ']
Getting the audio features for chunk:  10
['0V3wPSX9ygBnCm8psDIegu' '41WQUSINanQHfhfKpFDs

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.868,0.5380,5,-8.603,1,0.1740,0.2690,0.000003,0.0901,0.732,99.968,audio_features,2IGMVunIBsBLtEQyoI1Mu7,spotify:track:2IGMVunIBsBLtEQyoI1Mu7,https://api.spotify.com/v1/tracks/2IGMVunIBsBL...,https://api.spotify.com/v1/audio-analysis/2IGM...,231750,4
1,0.212,0.0375,9,-21.159,0,0.0413,0.9920,0.939000,0.1110,0.120,63.513,audio_features,7HNAsAfrpuY7kSbi1Kuk29,spotify:track:7HNAsAfrpuY7kSbi1Kuk29,https://api.spotify.com/v1/tracks/7HNAsAfrpuY7...,https://api.spotify.com/v1/audio-analysis/7HNA...,90642,4
2,0.918,0.6340,10,-5.176,1,0.3640,0.1710,0.000000,0.0819,0.932,135.082,audio_features,1ZWEehHLyzfkYn0jVslHCy,spotify:track:1ZWEehHLyzfkYn0jVslHCy,https://api.spotify.com/v1/tracks/1ZWEehHLyzfk...,https://api.spotify.com/v1/audio-analysis/1ZWE...,130889,4
3,0.666,0.8710,1,-6.017,0,0.0272,0.2700,0.020800,0.2630,0.936,108.501,audio_features,2EGaDf0cPX789H3LNeB03D,spotify:track:2EGaDf0cPX789H3LNeB03D,https://api.spotify.com/v1/tracks/2EGaDf0cPX78...,https://api.spotify.com/v1/audio-analysis/2EGa...,215326,4
4,0.603,0.4910,1,-8.745,1,0.0520,0.0212,0.000007,0.1120,0.355,134.950,audio_features,7pjPeCHhB5gvwONDGvNtH2,spotify:track:7pjPeCHhB5gvwONDGvNtH2,https://api.spotify.com/v1/tracks/7pjPeCHhB5gv...,https://api.spotify.com/v1/audio-analysis/7pjP...,260435,4


concat dataframe with song name id and datatframe with auido features

In [24]:
# concat songs dataframe with audio features

top_100_with_audio_features = pd.concat([top_100, df_audio_features_top_100], axis=1)
display(top_100_with_audio_features.head())
# saving concat file
top_100_with_audio_features.to_csv(r"./top_100_with_audio_features.csv", index=False) 

,Songs,Artists,id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Paint The Town Red,Doja Cat,[2IGMVunIBsBLtEQyoI1Mu7],0.868,0.5380,5,-8.603,1,0.1740,0.2690,...,0.0901,0.732,99.968,audio_features,2IGMVunIBsBLtEQyoI1Mu7,spotify:track:2IGMVunIBsBLtEQyoI1Mu7,https://api.spotify.com/v1/tracks/2IGMVunIBsBL...,https://api.spotify.com/v1/audio-analysis/2IGM...,231750,4
1,Snooze,SZA,[7HNAsAfrpuY7kSbi1Kuk29],0.212,0.0375,9,-21.159,0,0.0413,0.9920,...,0.1110,0.120,63.513,audio_features,7HNAsAfrpuY7kSbi1Kuk29,spotify:track:7HNAsAfrpuY7kSbi1Kuk29,https://api.spotify.com/v1/tracks/7HNAsAfrpuY7...,https://api.spotify.com/v1/audio-analysis/7HNA...,90642,4
2,Fast Car,Luke Combs,[1ZWEehHLyzfkYn0jVslHCy],0.918,0.6340,10,-5.176,1,0.3640,0.1710,...,0.0819,0.932,135.082,audio_features,1ZWEehHLyzfkYn0jVslHCy,spotify:track:1ZWEehHLyzfkYn0jVslHCy,https://api.spotify.com/v1/tracks/1ZWEehHLyzfk...,https://api.spotify.com/v1/audio-analysis/1ZWE...,130889,4
3,Cruel Summer,Taylor Swift,[2EGaDf0cPX789H3LNeB03D],0.666,0.8710,1,-6.017,0,0.0272,0.2700,...,0.2630,0.936,108.501,audio_features,2EGaDf0cPX789H3LNeB03D,spotify:track:2EGaDf0cPX789H3LNeB03D,https://api.spotify.com/v1/tracks/2EGaDf0cPX78...,https://api.spotify.com/v1/audio-analysis/2EGa...,215326,4
4,I Remember Everything,Zach Bryan Featuring Kacey Musgraves,[7pjPeCHhB5gvwONDGvNtH2],0.603,0.4910,1,-8.745,1,0.0520,0.0212,...,0.1120,0.355,134.950,audio_features,7pjPeCHhB5gvwONDGvNtH2,spotify:track:7pjPeCHhB5gvwONDGvNtH2,https://api.spotify.com/v1/tracks/7pjPeCHhB5gv...,https://api.spotify.com/v1/audio-analysis/7pjP...,260435,4


Create a function get_audio_features(list_of_song_ids) to obtain the audio features of a given list of songs (the content of list_of_songs can be the href/id/uri or a list with a single song IDs).

In [6]:
# Read the CSV file into a DataFrame
not_top_100 = pd.read_csv("not_top_100.csv")

# Create a new DataFrame with only the "song_name" column
not_top_100 = not_top_100[['song_name']].copy()

# Rename the "song_name" column to "Songs"
not_top_100.rename(columns={'song_name': 'Songs'}, inplace=True)

# Display the first few rows of the modified DataFrame
display(not_top_100.head())

,Songs
0,Live Television
1,Beast Mode (feat. PnB Rock & YoungBoy Never Br...
2,Make It Rain
3,No Opp Left Behind
4,Storyboard for a Pessimist


In [12]:
#display(nontop_100.head())
not_top_100 = not_top_100.sample(1000 , replace = True)
list_of_ids=[]
for song in not_top_100['Songs']:
    try:
        id = search_song(song, "", 1)
        list_of_ids.append(id)
    except:
        print("Song not found!")
        list_of_ids.append("NONE")

list_of_ids = [ item[0] for item in list_of_ids]        
not_top_100["id"] = list_of_ids
display(not_top_100.head())
not_top_100=not_top_100[not_top_100['id']!="NONE"]
#pd.to_csv("not_top_100_with_id.csv")
not_top_100.to_csv(r"./not_top_100_with_id.csv", index=False)



,Songs,id
3413,"New Chains, Same Shackles",4HSX752K4acry8NQ0ejySN
4945,Altijd Wel Iemand,71efnjRKag5lHRjYuf2BIU
1455,No More,5rkXczEaROyDNylcK9sAZ6
3642,My Happy Ending,6sqNctd7MlJoKDOxPVCAvU
3102,Ambitionz Az A Ridah,5g9lS8deSIxItFBmZRC4vN


In [13]:
not_top_100.head()

,Songs,id
3413,"New Chains, Same Shackles",4HSX752K4acry8NQ0ejySN
4945,Altijd Wel Iemand,71efnjRKag5lHRjYuf2BIU
1455,No More,5rkXczEaROyDNylcK9sAZ6
3642,My Happy Ending,6sqNctd7MlJoKDOxPVCAvU
3102,Ambitionz Az A Ridah,5g9lS8deSIxItFBmZRC4vN


In [15]:
list_of_ids

['4HSX752K4acry8NQ0ejySN',
 '71efnjRKag5lHRjYuf2BIU',
 '5rkXczEaROyDNylcK9sAZ6',
 '6sqNctd7MlJoKDOxPVCAvU',
 '5g9lS8deSIxItFBmZRC4vN',
 '3tQgnyZFWXXpMnDl66kfDS',
 '32j7Enac4CN6qkA6OAamsQ',
 '4jcQEjvPOz8viU3cPuHvrV',
 '5XL1A1u2h0soVQtoUa8NdS',
 '0DBtpmV5CgKgEkuEVciw7P',
 '4odoLyHnSx2jIcXs16IvsV',
 '1lnW6hdkIDZU6JA53R0emi',
 '0ZQmpf5gk4k5VMopcAtyAc',
 '0xeeMGCf3w8Emm6hpYVVgK',
 '50kcRtu673zyBLtvvRMKHg',
 '3HbreCF19wV7JsqoFRM4SH',
 '5x1C3AAQyeMXfpQq0P07am',
 '4hfIVhq0F0zFUcrbecsYmo',
 '7sVmb63Km2TWwCwOeVn57M',
 '32PvPvfwJwY7ecoAG2LTyq',
 '3r2vyNnqFKr6IraCqLtoBI',
 '2ExyV1cm9MLVze99wLKCuv',
 '7jzUgYVnL1fHdj8wexTkYV',
 '6FT9FZccFD6nE8dMNslz2n',
 '3Gf5nttwcX9aaSQXRWidEZ',
 '2tU1xxr1ce4HrAt5l3YtQ1',
 '0725YWm6Z0TpZ6wrNk64Eb',
 '32FhTAAjKZaBein150o3Px',
 '7lGq7I6xre0c7uUIwg7aB5',
 '429IbFR4yp2J81CeTwF5iY',
 '0AFydkgl5Creero3gNjqvF',
 '1LBVq13HVunow3ofNJQ8ea',
 '1RNtm45kw0hPMBz7gKiIYu',
 '5vPUaojZGK6US3FRZYdQfM',
 '1sPNUO5QgfjRFYauCmelNb',
 '5eWYhusHCbD6GQS9Ctac6I',
 '7dLGc4FFTotdJMXJCJxcki',
 

In [17]:
df_audio_features_not_top_100 = get_audio_features(list_of_ids)
display(df_audio_features_not_top_100.head())
df_audio_features_not_top_100.to_csv(r"./df_audio_features_not_top_100.csv", index=False) 



Total Chunk Count: 50
['4HSX752K4acry8NQ0ejySN' '71efnjRKag5lHRjYuf2BIU'
 '5rkXczEaROyDNylcK9sAZ6' '6sqNctd7MlJoKDOxPVCAvU'
 '5g9lS8deSIxItFBmZRC4vN' '3tQgnyZFWXXpMnDl66kfDS'
 '32j7Enac4CN6qkA6OAamsQ' '4jcQEjvPOz8viU3cPuHvrV'
 '5XL1A1u2h0soVQtoUa8NdS' '0DBtpmV5CgKgEkuEVciw7P'
 '4odoLyHnSx2jIcXs16IvsV' '1lnW6hdkIDZU6JA53R0emi'
 '0ZQmpf5gk4k5VMopcAtyAc' '0xeeMGCf3w8Emm6hpYVVgK'
 '50kcRtu673zyBLtvvRMKHg' '3HbreCF19wV7JsqoFRM4SH'
 '5x1C3AAQyeMXfpQq0P07am' '4hfIVhq0F0zFUcrbecsYmo'
 '7sVmb63Km2TWwCwOeVn57M' '32PvPvfwJwY7ecoAG2LTyq']
Getting the audio features for chunk:  1
['3r2vyNnqFKr6IraCqLtoBI' '2ExyV1cm9MLVze99wLKCuv'
 '7jzUgYVnL1fHdj8wexTkYV' '6FT9FZccFD6nE8dMNslz2n'
 '3Gf5nttwcX9aaSQXRWidEZ' '2tU1xxr1ce4HrAt5l3YtQ1'
 '0725YWm6Z0TpZ6wrNk64Eb' '32FhTAAjKZaBein150o3Px'
 '7lGq7I6xre0c7uUIwg7aB5' '429IbFR4yp2J81CeTwF5iY'
 '0AFydkgl5Creero3gNjqvF' '1LBVq13HVunow3ofNJQ8ea'
 '1RNtm45kw0hPMBz7gKiIYu' '5vPUaojZGK6US3FRZYdQfM'
 '1sPNUO5QgfjRFYauCmelNb' '5eWYhusHCbD6GQS9Ctac6I'
 '7dLGc4FFTotdJMXJ

['19VTqucnxJuH6kilNdweTh' '12yeD2FKVnJ5KjRL8I8DG9'
 '6DVOZMNJku6bScCLYTWAUH' '5fbWyPoWVsQMzEsSKOYdq4'
 '0jY3JksZbbKdbRb1ZpadtT' '6GsZH1C280FUJXY5Dh0T7b'
 '1LBVq13HVunow3ofNJQ8ea' '5x1C3AAQyeMXfpQq0P07am'
 '78QR3Wp35dqAhFEc2qAGjE' '450i5pA9x18RiXiK4WCvpS'
 '41QEKSIHQh8YOZ9N9tQeLu' '6KKZMSxKhHiAHpDIWV9fU1'
 '33G3DmW97fM5rby7t7Eb0w' '0jdGv5W4ta82JZCVmUHKn2'
 '0725YWm6Z0TpZ6wrNk64Eb' '5cvp5GwijJNiS5s1a8145Q'
 '0DBtpmV5CgKgEkuEVciw7P' '2c6XGY8oaMu6c0u0ggP81q'
 '4Csbgu7LMDGPLjhPgHfQmZ' '5IUtvfNvOyVYZUa6AJFrnP']
Getting the audio features for chunk:  16
['4E2SrUpRFBtDWMKlWvceta' '08a8s9icDsEVU7RcY5FFLD'
 '1lD0BXIEtkVfsKIsOswz5R' '1utcAgwhvomMzgSNo0HDoN'
 '6vQ8FrWAzwTMIvjaDyuqu4' '1EqTGC1pfGHm4zCxgxPmL3'
 '2ExyV1cm9MLVze99wLKCuv' '0vqOcnyTzyCjmsoF51Ma4F'
 '4Csbgu7LMDGPLjhPgHfQmZ' '4AMqQJ7Gcp4nxTtXfaa2N2'
 '4QjaftuMbs4BBLucS7RH5j' '2Quf5SaypvUGMMjWTZ7gHT'
 '6FoWmEdb1WmsKckkZ4Rw5f' '7BMO7O7ImjV8HNTH74Tshv'
 '4Dq749x2QP6OXTURJ9GGY8' '5IUtvfNvOyVYZUa6AJFrnP'
 '1iKjPM2YygFxUqjUakAI4n' '75pMEcHJjLRv

['4imphpMfiSMhUIlJMkOHaI' '0jdGv5W4ta82JZCVmUHKn2'
 '0dAeZUKHCo70CusGsluvDz' '5KLhTEwodiq7etg8aQHFO5'
 '13qQO4TykIyDdd4Nw5vfc0' '5yhFMyyzQKl9w3D4oWXPPt'
 '3mckyat5lAxzBWCijrrOl8' '2jspAMff1XSSzkYhZJtzZV'
 '03eJ2DclFWXYU8GWgANdmZ' '3j3USJELvS2U1rXvKgphAw'
 '68osIGtVjM7QWVe6pazLHj' '4x7D71tOwMiEDunEr2sgTZ'
 '7IlCTvmleZjYIZebMZSa0K' '54kn0cP5ttd5endujuWkZ1'
 '0QmiOEUYqhX0wStLFLqM9o' '0wqeaDeaKq14IuQESPqBEJ'
 '50R1kzup8Unwb67AYMOe3c' '0fCwrjKfWxF3xQhZrYUw4T'
 '4dVbhS6OiYvFikshyaQaCN' '42DZT2eoL7rJBw21f3sb0h']
Getting the audio features for chunk:  31
['329ZgIFH3PVqiUjEnwApBP' '6l7VxGCmqeseT0K0L9tRXh'
 '2Gyc6e2cLxA5hoX1NOvYnU' '0xeeMGCf3w8Emm6hpYVVgK'
 '3j3USJELvS2U1rXvKgphAw' '3OwYEJd5C6V39YlwEZeuHz'
 '797i2RDE7Dpx8CHwo5R4mG' '4FzMYYcuQg10kBzXgauodD'
 '621BOqoCBatLdmnzyEWrnP' '3NEVsu4jLu0aTTGtTFswL4'
 '48WJUYrl91l90PZ86pkqqV' '23oxJmDc1V9uLUSmN2LIvx'
 '5WBdb9l4sMBv2mU7k8Psen' '6z6W7S93PQsoe6xB4BIgyy'
 '2ZgbH7uAjP9tbBOhsG4H44' '1iKjPM2YygFxUqjUakAI4n'
 '21vyka5NbqVfWcdURdoF4f' '5fQDGI8le5b8

['2jdAqci2mLt0Njv8euSZWS' '5wewATXAUVA7g9Ivynb0bA'
 '2lXHg7BtTNoP39AezyCSfL' '5rkXczEaROyDNylcK9sAZ6'
 '2lYAMoHSQmfFyYyEj9GuK5' '7GiozRoMk95aFl1WbrDdjX'
 '5HJ7FBXsXfUF4XX6Oqz1DE' '5L0rI8ZZqRBicQ24PIoSXm'
 '5OFOaDl2LnR1Y6JnfdQMXm' '5GHGsPWg1RGPrV93VAigKc'
 '2MF4rIXGZ5s12KI49gAKxQ' '40Fmr4mXhz4PtrMAPTnoXB'
 '4cJsJdwEp3y4K1slV5fRZm' '0xeeMGCf3w8Emm6hpYVVgK'
 '1TMD9dsPs6s16QyPa1VqS4' '7kekN35Gq6zbC2o0yLyWxi'
 '0lB24x4SNbi8lLhUAsdoMO' '6qRofeOAJif90e0IPl9gz4'
 '5KhFaq45chTw8RGfWo8T8J' '6n3HGiq4v35D6eFOSwqYuo']
Getting the audio features for chunk:  46
['3NqcLK0DnlKbK0rTG1Ngd0' '4ncXLUcIbgwp2Bm9rGzohd'
 '5yMyLZu4wPvWcbLFiRhYVM' '1lD0BXIEtkVfsKIsOswz5R'
 '4AMqQJ7Gcp4nxTtXfaa2N2' '4VbhCvfzoMG1q3WpUclxl7'
 '2QN9zL3SzXdGqqmXO68gFp' '0AKAxdNkwq9ZxRdW1DN9zW'
 '32j7Enac4CN6qkA6OAamsQ' '5M7OvJqw8FvIr4jLJLwCi1'
 '7roOWd3X0wNapxow88nrSv' '3HbreCF19wV7JsqoFRM4SH'
 '7L4G39PVgMfaeHRyi1ML7y' '3MLv4atd5U56jNq3Nh1IFn'
 '2SGZHJNldk2WWrmIV5LOkp' '0M4QpC8pNQ4QO2uUzeZIVD'
 '6Jy0O6u2vRtFdszpirj7K0' '7Hvy3mngoXxY

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.917,0.494,1,-6.972,1,0.4550,0.06230,0.000000,0.1060,0.530,130.113,audio_features,4HSX752K4acry8NQ0ejySN,spotify:track:4HSX752K4acry8NQ0ejySN,https://api.spotify.com/v1/tracks/4HSX752K4acr...,https://api.spotify.com/v1/audio-analysis/4HSX...,191744,4
1,0.722,0.612,6,-11.323,1,0.4860,0.71100,0.000001,0.1390,0.935,78.330,audio_features,71efnjRKag5lHRjYuf2BIU,spotify:track:71efnjRKag5lHRjYuf2BIU,https://api.spotify.com/v1/tracks/71efnjRKag5l...,https://api.spotify.com/v1/audio-analysis/71ef...,129498,4
2,0.613,0.772,7,-6.670,1,0.3220,0.00070,0.148000,0.7270,0.428,119.922,audio_features,5rkXczEaROyDNylcK9sAZ6,spotify:track:5rkXczEaROyDNylcK9sAZ6,https://api.spotify.com/v1/tracks/5rkXczEaROyD...,https://api.spotify.com/v1/audio-analysis/5rkX...,158000,4
3,0.414,0.936,2,-2.407,1,0.0758,0.00136,0.000000,0.3690,0.740,170.229,audio_features,6sqNctd7MlJoKDOxPVCAvU,spotify:track:6sqNctd7MlJoKDOxPVCAvU,https://api.spotify.com/v1/tracks/6sqNctd7MlJo...,https://api.spotify.com/v1/audio-analysis/6sqN...,242413,4
4,0.801,0.910,2,-6.566,1,0.2250,0.00239,0.000891,0.0737,0.805,87.277,audio_features,5g9lS8deSIxItFBmZRC4vN,spotify:track:5g9lS8deSIxItFBmZRC4vN,https://api.spotify.com/v1/tracks/5g9lS8deSIxI...,https://api.spotify.com/v1/audio-analysis/5g9l...,278387,4


In [19]:
not_top_100_with_audio_features = pd.concat([not_top_100.reset_index(drop=True), df_audio_features_not_top_100.reset_index(drop=True)], axis=1)
display(not_top_100_with_audio_features.head())
not_top_100_with_audio_features.to_csv(r"./not_top_100_with_audio_features.csv", index=False)


,Songs,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,"New Chains, Same Shackles",4HSX752K4acry8NQ0ejySN,0.917,0.494,1,-6.972,1,0.4550,0.06230,0.000000,0.1060,0.530,130.113,audio_features,4HSX752K4acry8NQ0ejySN,spotify:track:4HSX752K4acry8NQ0ejySN,https://api.spotify.com/v1/tracks/4HSX752K4acr...,https://api.spotify.com/v1/audio-analysis/4HSX...,191744,4
1,Altijd Wel Iemand,71efnjRKag5lHRjYuf2BIU,0.722,0.612,6,-11.323,1,0.4860,0.71100,0.000001,0.1390,0.935,78.330,audio_features,71efnjRKag5lHRjYuf2BIU,spotify:track:71efnjRKag5lHRjYuf2BIU,https://api.spotify.com/v1/tracks/71efnjRKag5l...,https://api.spotify.com/v1/audio-analysis/71ef...,129498,4
2,No More,5rkXczEaROyDNylcK9sAZ6,0.613,0.772,7,-6.670,1,0.3220,0.00070,0.148000,0.7270,0.428,119.922,audio_features,5rkXczEaROyDNylcK9sAZ6,spotify:track:5rkXczEaROyDNylcK9sAZ6,https://api.spotify.com/v1/tracks/5rkXczEaROyD...,https://api.spotify.com/v1/audio-analysis/5rkX...,158000,4
3,My Happy Ending,6sqNctd7MlJoKDOxPVCAvU,0.414,0.936,2,-2.407,1,0.0758,0.00136,0.000000,0.3690,0.740,170.229,audio_features,6sqNctd7MlJoKDOxPVCAvU,spotify:track:6sqNctd7MlJoKDOxPVCAvU,https://api.spotify.com/v1/tracks/6sqNctd7MlJo...,https://api.spotify.com/v1/audio-analysis/6sqN...,242413,4
4,Ambitionz Az A Ridah,5g9lS8deSIxItFBmZRC4vN,0.801,0.910,2,-6.566,1,0.2250,0.00239,0.000891,0.0737,0.805,87.277,audio_features,5g9lS8deSIxItFBmZRC4vN,spotify:track:5g9lS8deSIxItFBmZRC4vN,https://api.spotify.com/v1/tracks/5g9lS8deSIxI...,https://api.spotify.com/v1/audio-analysis/5g9l...,278387,4


 split the list of song IDs into "chunks" of 50 song IDs and wait 20 seconds before asking for the audio features of the next "chunk" (for your own peace of mind add a "print("Collecting IDs for chunk...") message to show the progress). To create chunks of song IDs consider using np.split